# SRL Patient recognition experiments

In this notebook I carry out experiments to test whether two Semantic Role Labelling (SRL) systems can correctly identify patients in sentences with varying structures. This code was based on code provided by Pia Sommerauer.

In this code I load two models, namely the AllenNLP SRL model and the AllenNLP SRL BERT model. I create a variety of tets cases, for wich I evaluate the performance of the two models. All the test sentences are stored in a json file specified through the `test_sents_path` variable. The SRL predictions are stored in the json file specified through `srl_pred_path`, and similarly the SRL BERT predictions are stored at the path `bert_pred_path`.

### Recognizing the difference between an instrument and a description
* Instrument: 'John hit the door with a hammer'
* Description : 'John hit the door with the poster'


In [4]:
from allennlp_models.pretrained import load_predictor

In [5]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [6]:
from checklist.pred_wrapper import PredictorWrapper

In [3]:
import json
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [1]:
from utils_functions import *

### Load the models 

In [7]:
# load the regular SRL model
srl_predictor = load_predictor('structured-prediction-srl')
# load the SRL BERT model
srlbert_predictor = load_predictor('structured-prediction-srl-bert')

2022-03-24 16:30:57,140 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-24 16:30:57,270 - INFO - allennlp.common.plugins - Plugin allennlp_semparse available
2022-03-24 16:30:57,429 - INFO - allennlp.common.plugins - Plugin allennlp_server available
2022-03-24 16:30:57,465 - INFO - allennlp.common.params - id = pair-classification-esim
2022-03-24 16:30:57,466 - INFO - allennlp.common.params - registered_model_name = esim
2022-03-24 16:30:57,466 - INFO - allennlp.common.params - model_class = None
2022-03-24 16:30:57,467 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-24 16:30:57,467 - INFO - allennlp.common.params - display_name = Enhanced LSTM for Natural Language Inference
2022-03-24 16:30:57,467 - INFO - allennlp.common.params - task_id = textual_entailment
2022-03-24 16:30:57,468 - INFO - allennlp.common.params - model_usage.archive_file = esim-elmo-2020.11.11.tar.gz
2022-03-24 16:30:57,469 - INFO - allennlp.common.params - mod

2022-03-24 16:30:57,574 - INFO - allennlp.common.params - metrics.decision_thresholds = None
2022-03-24 16:30:57,575 - INFO - allennlp.common.params - metrics.variation_approaches = None
2022-03-24 16:30:57,576 - INFO - allennlp.common.params - evaluation_data.dataset = None
2022-03-24 16:30:57,577 - INFO - allennlp.common.params - evaluation_data.motivation = None
2022-03-24 16:30:57,578 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-24 16:30:57,585 - INFO - allennlp.common.params - training_data.dataset.name = DROP
2022-03-24 16:30:57,586 - INFO - allennlp.common.params - training_data.dataset.url = https://allennlp.org/drop
2022-03-24 16:30:57,587 - INFO - allennlp.common.params - training_data.motivation = None
2022-03-24 16:30:57,587 - INFO - allennlp.common.params - training_data.preprocessing = None
2022-03-24 16:30:57,589 - INFO - allennlp.common.params - quantitative_analyses.unitary_results = None
2022-03-24 16:30:57,590 - INFO - allennlp.commo

2022-03-24 16:30:57,739 - INFO - allennlp.common.params - model_details.version = 1
2022-03-24 16:30:57,740 - INFO - allennlp.common.params - model_details.model_type = RoBERTa large
2022-03-24 16:30:57,741 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Liu2019RoBERTaAR,
title={RoBERTa: A Robustly Optimized BERT Pretraining Approach},
author={Y. Liu and Myle Ott and Naman Goyal and Jingfei Du and Mandar Joshi and Danqi Chen and Omer Levy and M. Lewis and Luke Zettlemoyer and Veselin Stoyanov},
journal={ArXiv},
year={2019},
volume={abs/1907.11692}}

2022-03-24 16:30:57,742 - INFO - allennlp.common.params - model_details.paper.title = RoBERTa: A Robustly Optimized BERT Pretraining Approach (Liu et al)
2022-03-24 16:30:57,742 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:198953378
2022-03-24 16:30:57,744 - INFO - allennlp.common.params - model_details.license = None
2022-03-24 16:30:57,745 - INFO - allennlp

2022-03-24 16:30:57,899 - INFO - allennlp.common.params - model_class = None
2022-03-24 16:30:57,899 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-24 16:30:57,900 - INFO - allennlp.common.params - display_name = Coreference Resolution
2022-03-24 16:30:57,901 - INFO - allennlp.common.params - task_id = coref
2022-03-24 16:30:57,902 - INFO - allennlp.common.params - model_usage.archive_file = coref-spanbert-large-2021.03.10.tar.gz
2022-03-24 16:30:57,903 - INFO - allennlp.common.params - model_usage.training_config = coref/coref_spanbert_large.jsonnet
2022-03-24 16:30:57,904 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.1.0 allennlp-models==2.1.0
2022-03-24 16:30:57,905 - INFO - allennlp.common.params - model_usage.overrides = None
2022-03-24 16:30:57,907 - INFO - allennlp.common.params - model_details.description = The basic outline of this model is to get an embedded representation of each span in the document.

2022-03-24 16:30:58,010 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:199453025
2022-03-24 16:30:58,011 - INFO - allennlp.common.params - model_details.license = None
2022-03-24 16:30:58,011 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-03-24 16:30:58,012 - INFO - allennlp.common.params - intended_use.primary_uses = This model is developed for the AllenNLP demo.
2022-03-24 16:30:58,013 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-03-24 16:30:58,014 - INFO - allennlp.common.params - intended_use.out_of_scope_use_cases = None
2022-03-24 16:30:58,018 - INFO - allennlp.common.params - factors.relevant_factors = None
2022-03-24 16:30:58,018 - INFO - allennlp.common.params - factors.evaluation_factors = None
2022-03-24 16:30:58,019 - INFO - allennlp.common.params - metrics.model_performance_measures = Accuracy and F1-score
2022-03-24 16:30:58,020 - INFO - allennlp.

...

2022-03-24 16:31:05,943 - INFO - allennlp.common.params - model_details.developed_by = Lample et al
2022-03-24 16:31:05,943 - INFO - allennlp.common.params - model_details.contributed_by = None
2022-03-24 16:31:05,944 - INFO - allennlp.common.params - model_details.date = 2020-06-24
2022-03-24 16:31:05,947 - INFO - allennlp.common.params - model_details.version = 1
2022-03-24 16:31:05,947 - INFO - allennlp.common.params - model_details.model_type = BiLSTM
2022-03-24 16:31:05,949 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Lample2016NeuralAF,
title={Neural Architectures for Named Entity Recognition},
author={Guillaume Lample and Miguel Ballesteros and Sandeep Subramanian and K. Kawakami and Chris Dyer},
journal={ArXiv},
year={2016},
volume={abs/1603.01360}}

2022-03-24 16:31:05,949 - INFO - allennlp.common.params - model_details.paper.title = Neural Architectures for Named Entity Recognition
2022-03-24 16:31:05,950 - INFO - allennlp.common.params - model_d

2022-03-24 16:31:06,060 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-24 16:31:06,061 - INFO - allennlp.common.params - training_data.dataset.name = CNN/DailyMail
2022-03-24 16:31:06,062 - INFO - allennlp.common.params - training_data.dataset.notes = Please download the data from the url provided.
2022-03-24 16:31:06,063 - INFO - allennlp.common.params - training_data.dataset.url = https://github.com/abisee/cnn-dailymail
2022-03-24 16:31:06,063 - INFO - allennlp.common.params - training_data.motivation = None
2022-03-24 16:31:06,064 - INFO - allennlp.common.params - training_data.preprocessing = None
2022-03-24 16:31:06,066 - INFO - allennlp.common.params - quantitative_analyses.unitary_results = None
2022-03-24 16:31:06,067 - INFO - allennlp.common.params - quantitative_analyses.intersectional_results = None
2022-03-24 16:31:06,068 - INFO - allennlp.common.params - model_ethical_considerations.ethical_considerations = None
2022-03-24 16:31:06,069 - INF

2022-03-24 16:31:26,343 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.LayerNorm.bias
2022-03-24 16:31:26,343 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.LayerNorm.weight
2022-03-24 16:31:26,344 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.dense.bias
2022-03-24 16:31:26,344 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.dense.weight
2022-03-24 16:31:26,345 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.key.bias
2022-03-24 16:31:26,345 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.key.weight
2022-03-24 16:31:26,345 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.query.bias
2022-03-24 16:31:26,345 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.query.weight
2022-03-24 16:31:26,346 - INFO - allennlp.nn.initial

2022-03-24 16:31:26,410 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.query.weight
2022-03-24 16:31:26,410 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.value.bias
2022-03-24 16:31:26,411 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.value.weight
2022-03-24 16:31:26,411 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.intermediate.dense.bias
2022-03-24 16:31:26,412 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.intermediate.dense.weight
2022-03-24 16:31:26,412 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.LayerNorm.bias
2022-03-24 16:31:26,413 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.LayerNorm.weight
2022-03-24 16:31:26,413 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.dense.bias
2022-03-24 16:31:26,414 - INFO - allennlp.nn.initializers -    bert_model.encoder.la

2022-03-24 16:31:26,470 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.7.output.dense.bias
2022-03-24 16:31:26,470 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.7.output.dense.weight
2022-03-24 16:31:26,471 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.LayerNorm.bias
2022-03-24 16:31:26,472 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.LayerNorm.weight
2022-03-24 16:31:26,477 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.dense.bias
2022-03-24 16:31:26,478 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.dense.weight
2022-03-24 16:31:26,479 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.key.bias
2022-03-24 16:31:26,480 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.key.weight
2022-03-24 16:31:26,480 - INFO - allennlp.nn.initializers -    bert_

In [8]:
#functions to create model predictions for a list containing sentences
### added by pia, edited by Goya ###

def predict_srl(data):
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred


def predict_srlbert(data):
    pred = []
    for d in data:
        pred.append(srlbert_predictor.predict(d))
    return pred

predict_srl = PredictorWrapper.wrap_predict(predict_srl)
predict_srlbert = PredictorWrapper.wrap_predict(predict_srlbert)

### Define output file paths

In [12]:
#create lists to store test sentences and model predictions in 
test_data = []
SRLBERT_predictions = []
SRL_predictions = []

In [9]:
#define paths to output files
test_sents_path = './JSON_test_and_predict_files/test_data_instrument.json'
bert_pred_path = './JSON_test_and_predict_files/BERT_predictions_instrument.json'
srl_pred_path = './JSON_test_and_predict_files/SRL_predictions_instrument.json'

#set name of current capability
capability = 'instrument_recognition'

### Load Checklist tests (Load functions defined in utils)
Load functions to test whether the arguments are classified correctly

In [2]:
expect_arg1_verb0 = Expect.single(found_door_arg1_verb0)
expect_arg2_verb0 = Expect.single(found_door_arg2_verb0)
expect_arg2_verb1 = Expect.single(found_door_arg2_verb1)

### Load wordlists to use in sample sentences

In [10]:
editor = Editor()
instrument = ['hammer', 'knife', 'stone', 'bottle', 'table', 'chair', 'racket']
description = ['poster', 'flag', 'window', 'bell', 'paint', 'holes', 'mailbox', 'nameplate']

## Agent recognition : small and large predicate distance

In [13]:
#create samples
testcase_name = 'instrument'
t = editor.template("{first_name} hit the door with the {instrument}", instrument=instrument, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg2_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    1 (1.0%)

Example fails:
[ARG0: Dick] [V: hit] [ARG1: the door] [ARGM-MNR: with the knife]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    17 (17.0%)

Example fails:
[ARG0: Harry] [V: hit] [ARG1: the door] [ARGM-MNR: with the racket]
----
[ARG0: Linda] [V: hit] [ARG1: the door] [ARGM-MNR: with the racket]
----
[ARG0: Steven] [V: hit] [ARG1: the door] [C-ARG1: with the table]
----


In [14]:
#create samples
testcase_name = 'patient'
t = editor.template("{first_name} hit the door with the {description}", description=description, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg1_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)


SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARG0: Howard] [V: hit] [ARG1: the door] [ARG2: with the window]
----
[ARG0: Alexandra] [V: hit] [ARG1: the door] [ARG2: with the mailbox]
----
[ARG0: Howard] [V: hit] [ARG1: the door] [ARG2: with the holes]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARG0: Howard] [V: hit] [ARG1: the door] [ARG2: with the window]
----
[ARG0: Bill] [V: hit] [ARG1: the door] [ARG2: with the bell]
----
[ARG0: Annie] [V: hit] [ARG1: the door] [ARG2: with the window]
----


### Store all data to JSON

In [15]:
#store the test sentences
store_data(test_sents_path, test_data, new_file=True)
#store the model predictions
store_data(bert_pred_path, SRLBERT_predictions, new_file=True)
store_data(srl_pred_path, SRL_predictions, new_file=True)